In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import random
from typing import List, Dict, Set, Tuple, Optional
from collections import defaultdict
import hashlib

class ShingleGenerator:
    """Generates shingles from input strings."""

    @staticmethod
    def generate_shingles(text: str, k: int = 4) -> Set[str]:
        """
        Generate k-shingles from a given text.

        Args:
            text: Input string to shingle
            k: Length of each shingle

        Returns:
            Set of unique shingles
        """
        text = text.lower().replace(" ", "")  # Case insensitive and remove spaces
        return {text[i:i+k] for i in range(len(text) - k + 1)}

class JaccardDistance:
    """Calculates Jaccard distance between sets."""

    @staticmethod
    def calculate(set1: Set, set2: Set) -> float:
        """
        Calculate Jaccard distance between two sets.

        Args:
            set1: First set
            set2: Second set

        Returns:
            Jaccard distance (1 - Jaccard similarity)
        """
        if not set1 and not set2:
            return 0.0
        intersection = len(set1 & set2)
        union = len(set1 | set2)
        return 1 - (intersection / union)

class AgglomerativeClustering:
    """Agglomerative hierarchical clustering for non-Euclidean spaces."""

    def __init__(self, min_clusters: int = 5, linkage: str = 'average'):
        """
        Initialize the clustering algorithm.

        Args:
            min_clusters: Stop when this many clusters remain
            linkage: Cluster distance method ('single', 'complete', 'average')
        """
        self.min_clusters = min_clusters
        self.linkage = linkage
        self.clusters_history = []

    def _initialize_clusters(self, data: List[Set[str]]) -> List[List[int]]:
        """Initialize each point as its own cluster."""
        return [[i] for i in range(len(data))]

    def _find_clustroid(self, cluster: List[int], distance_matrix: np.ndarray) -> int:
        """Find the clustroid (point with minimal average distance to others)."""
        cluster_distances = distance_matrix[cluster][:, cluster]
        avg_distances = cluster_distances.mean(axis=1)
        return cluster[np.argmin(avg_distances)]

    def _compute_cluster_distance(
        self,
        cluster1: List[int],
        cluster2: List[int],
        distance_matrix: np.ndarray
    ) -> float:
        """Compute distance between two clusters based on linkage method."""
        submatrix = distance_matrix[np.ix_(cluster1, cluster2)]

        if self.linkage == 'single':
            return np.min(submatrix)
        elif self.linkage == 'complete':
            return np.max(submatrix)
        elif self.linkage == 'average':
            return np.mean(submatrix)
        else:
            raise ValueError(f"Unknown linkage method: {self.linkage}")

    def fit(self, data: List[Set[str]]) -> List[List[int]]:
        """
        Perform hierarchical clustering.

        Args:
            data: List of sets (shingle sets for each string)

        Returns:
            List of clusters (each cluster is a list of indices)
        """
        n_samples = len(data)
        if n_samples == 0:
            return []

        # Precompute distance matrix
        distance_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(i+1, n_samples):
                distance = JaccardDistance.calculate(data[i], data[j])
                distance_matrix[i, j] = distance
                distance_matrix[j, i] = distance

        # Initialize each point as its own cluster
        clusters = self._initialize_clusters(data)
        self.clusters_history = [clusters.copy()]

        # Perform hierarchical clustering
        while len(clusters) > self.min_clusters:
            # Find the closest pair of clusters
            min_distance = float('inf')
            cluster1, cluster2 = -1, -1

            for i in range(len(clusters)):
                for j in range(i+1, len(clusters)):
                    distance = self._compute_cluster_distance(
                        clusters[i], clusters[j], distance_matrix
                    )
                    if distance < min_distance:
                        min_distance = distance
                        cluster1, cluster2 = i, j

            # Merge the two closest clusters
            new_cluster = clusters[cluster1] + clusters[cluster2]

            # Update clusters list
            if cluster1 < cluster2:
                del clusters[cluster2]
                del clusters[cluster1]
            else:
                del clusters[cluster1]
                del clusters[cluster2]

            clusters.append(new_cluster)
            self.clusters_history.append(clusters.copy())

        return clusters

    def get_cluster_stats(self, clusters: List[List[int]], distance_matrix: np.ndarray) -> Dict[int, float]:
        """
        Compute average distance of each point to its clustroid for each cluster.

        Args:
            clusters: List of clusters (each cluster is a list of indices)
            distance_matrix: Precomputed distance matrix

        Returns:
            Dictionary mapping cluster index to average distance
        """
        cluster_stats = {}

        for i, cluster in enumerate(clusters):
            if not cluster:
                continue

            clustroid = self._find_clustroid(cluster, distance_matrix)
            distances = distance_matrix[clustroid, cluster]
            avg_distance = np.mean(distances)
            cluster_stats[i] = avg_distance

        return cluster_stats

def generate_random_string(min_len: int = 32, max_len: int = 64) -> str:
    """
    Generate a random alphabetical string of random length.

    Args:
        min_len: Minimum string length
        max_len: Maximum string length

    Returns:
        Randomly generated string
    """
    length = random.randint(min_len, max_len)
    return ''.join(random.choices(string.ascii_letters, k=length)).lower()

def generate_dataset(n_samples: int = 10000) -> pd.DataFrame:
    """
    Generate a dataset of random strings and their shingles.

    Args:
        n_samples: Number of strings to generate

    Returns:
        DataFrame with index, string, and shingles columns
    """
    data = []

    for i in range(n_samples):
        s = generate_random_string()
        shingles = ShingleGenerator.generate_shingles(s)
        data.append({'index': i, 'string': s, 'shingles': shingles})

    return pd.DataFrame(data)

def plot_cluster_stats(cluster_stats: Dict[int, float]):
    """
    Plot a bar chart of cluster average distances.

    Args:
        cluster_stats: Dictionary mapping cluster index to average distance
    """
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(cluster_stats)), list(cluster_stats.values()))
    plt.xlabel('Cluster Index')
    plt.ylabel('Average Distance to Clustroid')
    plt.title('Cluster Quality Metrics')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def main():
    # Step 1: Generate dataset
    print("Generating dataset...")
    df = generate_dataset(10000)

    # Save to CSV
    df.to_csv('string_shingles_dataset.csv', index=False)
    print("Dataset saved to 'string_shingles_dataset.csv'")

    # For efficiency, we'll work with a sample of the data
    sample_size = 500  # Reduced for demonstration
    sample_data = df.sample(sample_size)['shingles'].tolist()

    # Step 2: Perform clustering
    print("Performing hierarchical clustering...")
    clusterer = AgglomerativeClustering(min_clusters=5, linkage='average')
    clusters = clusterer.fit(sample_data)

    # Step 3: Precompute distance matrix for cluster stats
    print("Computing cluster statistics...")
    distance_matrix = np.zeros((sample_size, sample_size))
    for i in range(sample_size):
        for j in range(i+1, sample_size):
            distance = JaccardDistance.calculate(sample_data[i], sample_data[j])
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance

    # Step 4: Compute and plot cluster stats
    cluster_stats = clusterer.get_cluster_stats(clusters, distance_matrix)
    plot_cluster_stats(cluster_stats)

if __name__ == "__main__":
    main()